In [1]:
import os
import re

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
from create_Tatoeba_train_test import get_language_mappings, get_sentence_word_char_len

In [3]:
from langid.langid import LanguageIdentifier, model

In [4]:
language_mappings = get_language_mappings("language_mappings.csv")
iso_639_2_English = language_mappings[["ISO 639-2", "English Name"]].set_index(keys="ISO 639-2")["English Name"].to_dict()
iso_639_2_iso_639_1 = language_mappings[["ISO 639-2", "ISO 639-1"]].set_index(keys="ISO 639-2")["ISO 639-1"].to_dict()

In [5]:
bin_labels = ["1","2","3","4","5","6","7","8","9","10","11 to 16", "17 to 27", "28 to 48", "49 to 99"]

In [6]:
%%time
counter = 0
for root, dirs, file in os.walk("output/"):
    if root == "output/":
        for i in file:
            train_sample = re.search(r"train", i)
            if train_sample:
                sample = pd.read_csv(f"output/{i}")
                sample["Language_English_Name"] = sample["Language"].map(iso_639_2_English)
                sample["Sentence_len"] = sample.apply(lambda row: get_sentence_word_char_len(row), axis=1)
                
                sample["Sentence_len_bin"] = pd.cut(
                    x=sample["Sentence_len"],
                    bins=[0,1,2,3,4,5,6,7,8,9,10,16,27,48,99],
                    labels=bin_labels
                )
                
                # find all languages to include in langid predictions
                set_languages = [iso_639_2_iso_639_1[i] for i in sample["Language"].unique().tolist()]
                lang_identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
                sample["langid"] = sample.apply(lambda row: list(lang_identifier.classify(row["Sentence"])), axis=1)
                sample["langid_language"], sample["langid_score"] = sample["langid"].str
                sample.drop(columns=["langid"], inplace=True)
                # map Language to English Name & langid_language to English Name
                ISO_639_2 = language_mappings.set_index("ISO 639-2")
                ISO_639_2 = ISO_639_2["English Name"].to_dict()
                sample["Language_English_Name"] = sample["Language"].map(ISO_639_2)

                ISO_639_1 = language_mappings.set_index("ISO 639-1")
                ISO_639_1 = ISO_639_1["English Name"].to_dict()
                sample["langid_language_English_Name"] = sample["langid_language"].map(ISO_639_1)
                
                counter += 1
                print(counter, i)
                display(sample.sample())

1 Tatoeba_stratify_train_2021-8-4_1139.csv


,Original Index,Sentence,Language,Language_English_Name,Sentence_len,Sentence_len_bin,langid_language,langid_score,langid_language_English_Name
20304,3170801,Se decía que el fuego había sido causado por u...,spa,Spanish,12,11 to 16,es,1.0,Spanish


2 Tatoeba_stratify_train_2021-8-4_1157.csv


,Original Index,Sentence,Language,Language_English_Name,Sentence_len,Sentence_len_bin,langid_language,langid_score,langid_language_English_Name
17645,1476013,Het viel me gelijk op dat ze bijzonder lange b...,nld,Dutch,11,11 to 16,nl,1.0,Dutch


Wall time: 4min 17s
